Github
https://github.com/Shadowjumper3000/ML-fundamentals-2025

# Bike Rental Analysis
This analysis explores patterns in bike rental data to understand key factors influencing rental behavior. The data is sourced from the UCI Machine Learning Repository and contains hourly rental data spanning two years. The analysis is structured as follows:


In [ ]:
# Check if running in Kaggle environment
import os
IN_KAGGLE = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

if IN_KAGGLE:
    # Kaggle-specific paths
    data_path = '/kaggle/input/bike-data/CapitalBikeSharing.csv'
    checkpoint_dir = '/kaggle/working/models'
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Create output directory for downloading models
    os.makedirs('/kaggle/working/output', exist_ok=True)

    print("Running in Kaggle environment.")
    print("Data will be loaded from:", data_path)
    print("Models will be saved to:", checkpoint_dir)
else:
    # Local paths
    data_path = '../data/hour.csv'
    checkpoint_dir = '../models/checkpoints'
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("Running in local environment.")
    print("Data will be loaded from:", data_path)
    print("Models will be saved to:", checkpoint_dir)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style for better visualizations
plt.style.use('default')
sns.set_theme()

# Load the dataset
hour_data = pd.read_csv(data_path)
print("Data loaded from" + data_path)

## 1. Initial Data Exploration
Let's examine the basic structure and statistics of our dataset.
AI was used to write the functions for the analysis.

In [ ]:
# Initial Data Exploration
print("Dataset Shape:", hour_data.shape)
print("\nDataset Info:")
print(hour_data.info())
print("\nDescriptive Statistics:")
print(hour_data.describe())

### 1.1 Target Variable Analysis
Analyzing the distribution of bike rentals (cnt) to understand the general rental patterns.

In [ ]:
# Target Variable Analysis (cnt)
plt.figure(figsize=(10, 6))
sns.histplot(hour_data['cnt'], kde=True)
plt.title('Distribution of Bike Rentals (cnt)')
plt.xlabel('Number of Rentals')
plt.ylabel('Frequency')
print("\nSkewness of cnt:", hour_data['cnt'].skew())

### 1.2 Temporal Pattern Analysis
Examining how rental patterns vary by hour and season.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Hour analysis
plt.figure(figsize=(12, 6))
sns.boxplot(x='hr', y='cnt', data=hour_data)
plt.title('Hourly Rental Pattern')
plt.xlabel('Hour of Day')
plt.ylabel('Number of Rentals')
plt.show()

# Rentals per year
# Calculate total rentals per year - safer approach
yearly_rentals = hour_data.groupby('yr')['cnt'].sum().reset_index()
yearly_rentals['year'] = yearly_rentals['yr'].map({0: '2011', 1: '2012'})

plt.figure(figsize=(8, 5))
ax = sns.barplot(x='year', y='cnt', data=yearly_rentals, 
                color=sns.color_palette("Blues_d")[3])
plt.title('Total Rentals Per Year')
plt.xlabel('Year')
plt.ylabel('Total Rentals')

# Add value labels on top of bars
for p in ax.patches:
    ax.annotate(f'{int(p.get_height()):,}', 
               (p.get_x() + p.get_width() / 2., p.get_height()),
               ha='center', va='bottom',
               xytext=(0, 5),
               textcoords='offset points')
plt.show()

# Rentals per month
# Calculate total rentals per month - more robust approach
monthly_rentals = hour_data.groupby('mnth')['cnt'].sum().reset_index()

plt.figure(figsize=(10, 6))
ax = sns.barplot(x='mnth', y='cnt', data=monthly_rentals, 
                color=sns.color_palette("Blues_d")[3])
plt.title('Total Rentals Per Month', pad=15)
plt.xlabel('Month')
plt.ylabel('Total Rentals')
plt.xticks(ticks=range(12), 
          labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], 
          rotation=45)

# Add value labels on top of bars
for p in ax.patches:
    ax.annotate(f'{int(p.get_height()):,}', 
               (p.get_x() + p.get_width() / 2., p.get_height()),
               ha='center', va='bottom',
               xytext=(0, 5),
               textcoords='offset points')

plt.tight_layout()
plt.show()

# Create a season-specific visualization to show the distribution
# Generate a seasonal visualization with average rentals
plt.figure(figsize=(12, 6))

# Calculate average rentals per hour by season
season_hourly_avg = hour_data.groupby(['season', 'hr'])['cnt'].mean().unstack()

# Map season numbers to names for better readability
season_names = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Fall'}
season_hourly_avg.index = [season_names[s] for s in season_hourly_avg.index]

# Plot hourly patterns by season
for season in season_hourly_avg.index:
    plt.plot(season_hourly_avg.columns, season_hourly_avg.loc[season], 
             label=season, marker='o', markersize=4, linewidth=2, alpha=0.7)

plt.title('Hourly Rental Patterns by Season', fontsize=16)
plt.xlabel('Hour of Day', fontsize=12)
plt.ylabel('Average Rentals', fontsize=12)
plt.xticks(range(0, 24, 2))
plt.grid(linestyle='--', alpha=0.7)
plt.legend(title='Season', loc='upper left')

# Highlight morning and evening peaks
plt.axvspan(7, 9, color='lightyellow', alpha=0.3, label='Morning Peak')
plt.axvspan(17, 19, color='lightblue', alpha=0.3, label='Evening Peak')
plt.tight_layout()
plt.show()

# Calculate and plot monthly total rentals by year
monthly_by_year = hour_data.groupby(['yr', 'mnth'])['cnt'].sum().unstack()
monthly_by_year.index = ['2011', '2012']  # Convert year index to readable format

plt.figure(figsize=(14, 6))
monthly_by_year.T.plot(kind='bar', figsize=(14, 6))
plt.title('Monthly Bike Rentals by Year', fontsize=16)
plt.xlabel('Month', fontsize=12)
plt.ylabel('Total Rentals', fontsize=12)
plt.xticks(ticks=range(12), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                                   'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.legend(title='Year')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
from scipy.stats import chi2_contingency
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Create a cross-tabulation between season and month
season_month_crosstab = pd.crosstab(
    hour_data['season'],
    hour_data['mnth']
)

# Perform chi-square test for independence
chi2, p, dof, expected = chi2_contingency(season_month_crosstab)

print(f"Chi2: {chi2:.2f}")
print(f"p-value: {p:.10f}")
print(f"DOF: {dof}")
print(f"Hypothesis: {'Rejected' if p < 0.05 else 'Not rejected'}")

# Compare correlation with target variable
print("\nCorrelation with target variable:")
print(f"Season-cnt correlation: {hour_data['season'].corr(hour_data['cnt']):.4f}")
print(f"Month-cnt correlation: {hour_data['mnth'].corr(hour_data['cnt']):.4f}")

# Check distribution of values
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.countplot(x='season', data=hour_data)
plt.title('Season Distribution')

plt.subplot(1, 2, 2)
sns.countplot(x='mnth', data=hour_data)
plt.title('Month Distribution')
plt.tight_layout()
plt.show()

As the 'Seasons' and 'mnth's are correlated, we will drop the 'mnth' column as this is less correlated with 'cnt' than 'seasons'.

In [ ]:
# Drop the mnth column as it's highly correlated with season
hour_data = hour_data.drop(columns=['mnth'])
print("Remaining columns:", hour_data.columns.tolist())

### 1.3 Holiday and Working Day Analysis
Investigating how holidays and working days affect rental patterns.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Holiday analysis at hourly level
holiday_labels = {0: 'Non-Holiday', 1: 'Holiday'}
sns.boxplot(x='holiday', y='cnt', data=hour_data, ax=ax1)
ax1.set_title('Hourly Rentals: Holiday vs Non-Holiday')
ax1.set_xticklabels([holiday_labels[i] for i in [0, 1]])
ax1.set_xlabel('Day Type')
ax1.set_ylabel('Hourly Rentals')

# Working day analysis at hourly level
workingday_labels = {0: 'Non-Working Day', 1: 'Working Day'}
sns.boxplot(x='workingday', y='cnt', data=hour_data, ax=ax2)
ax2.set_title('Hourly Rentals: Working vs Non-Working Days')
ax2.set_xticklabels([workingday_labels[i] for i in [0, 1]])
ax2.set_xlabel('Day Type')
ax2.set_ylabel('Hourly Rentals')

plt.tight_layout()

In [ ]:
# Create day of week visualization
plt.figure(figsize=(12, 8))

# Calculate average rentals by day of week
weekday_rentals = hour_data.groupby('weekday')['cnt'].mean().reset_index()

# Map weekday numbers to names for better readability
weekday_names = {
    0: 'Sunday',
    1: 'Monday',
    2: 'Tuesday',
    3: 'Wednesday',
    4: 'Thursday',
    5: 'Friday',
    6: 'Saturday'
}
weekday_rentals['day_name'] = weekday_rentals['weekday'].map(weekday_names)

# Create bar plot with improved styling
ax = sns.barplot(x='day_name', y='cnt', data=weekday_rentals, 
                order=[weekday_names[i] for i in range(7)],
                palette='Blues_d')
plt.title('Average Hourly Bike Rentals by Day of Week', fontsize=16, pad=20)
plt.xlabel('Day of Week', fontsize=14)
plt.ylabel('Average Hourly Rentals', fontsize=14)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add value labels on top of bars
for p in ax.patches:
    ax.annotate(f'{p.get_height():.1f}', 
               (p.get_x() + p.get_width() / 2., p.get_height()),
               ha='center', va='bottom',
               xytext=(0, 5),
               textcoords='offset points')

# Add weekday vs weekend comparison
weekday_mask = hour_data['weekday'].isin([1, 2, 3, 4, 5])
weekend_mask = hour_data['weekday'].isin([0, 6])
weekday_avg = hour_data[weekday_mask]['cnt'].mean()
weekend_avg = hour_data[weekend_mask]['cnt'].mean()
diff_pct = ((weekday_avg - weekend_avg) / weekend_avg) * 100

In [ ]:
# Analyze weekday effect on bike rentals (cnt) with hourly patterns by day
plt.figure(figsize=(14, 10))

# Create hourly patterns by day of week
hourly_weekday_data = hour_data.pivot_table(
    values='cnt', 
    index='hr', 
    columns='weekday', 
    aggfunc='mean'
)

# Map weekday numbers to names for better readability
hourly_weekday_data.columns = [weekday_names[day] for day in hourly_weekday_data.columns]

# Plot hourly patterns by day of week
for day in hourly_weekday_data.columns:
    plt.plot(hourly_weekday_data.index, hourly_weekday_data[day], 
             label=day, marker='o', markersize=5, alpha=0.7)

# Add highlights for peak hours
plt.axvspan(7, 9, color='lightblue', alpha=0.3, label='Morning Peak')
plt.axvspan(16, 19, color='lightyellow', alpha=0.3, label='Evening Peak')

plt.title('Hourly Rental Patterns by Day of Week', fontsize=16)
plt.xlabel('Hour of Day', fontsize=12)
plt.ylabel('Average Rentals', fontsize=12)
plt.xticks(range(0, 24))
plt.grid(linestyle='--', alpha=0.7)
plt.legend(title='Day of Week', loc='upper center', bbox_to_anchor=(0.5, -0.15), 
           fancybox=True, shadow=True, ncol=4)

# Calculate and display correlation between weekday and cnt
weekday_cnt_corr = hour_data['weekday'].corr(hour_data['cnt'])
plt.annotate(f'Correlation between weekday and cnt: {weekday_cnt_corr:.3f}',
            xy=(0.5, 0.97), xycoords='axes fraction',
            ha='center', fontsize=12,
            bbox=dict(boxstyle='round,pad=0.5', fc='white', alpha=0.8))
plt.annotate(f'Correlation between workingday and cnt: {hour_data["workingday"].corr(hour_data["cnt"]):.3f}',
            xy=(0.5, 0.92), xycoords='axes fraction',
            ha='center', fontsize=12,
            bbox=dict(boxstyle='round,pad=0.5', fc='white', alpha=0.8))
plt.annotate(f'Correlation between holiday and cnt: {hour_data["holiday"].corr(hour_data["cnt"]):.3f}',
            xy=(0.5, 0.87), xycoords='axes fraction',
            ha='center', fontsize=12,
            bbox=dict(boxstyle='round,pad=0.5', fc='white', alpha=0.8))

plt.tight_layout()
plt.show()

Due to the spread of rentals on weekday vs weekend, we will drop the 'weekday' column as it is less correlated with 'cnt' than 'workingday'. Additionally for model simplicity it only requires learning binary classification of work / !work days. Rather than learning 7 classes of weekdays.

In [ ]:
# Drop the weekday column as it's less correlated with 'cnt' than 'workingday'
hour_data = hour_data.drop(columns=['weekday'])
print(f"'weekday' column dropped. Remaining columns: {hour_data.columns.tolist()}")

In [ ]:
import scipy.stats as stats

# Calculate basic statistics for workingday vs cnt
workingday_stats = hour_data.groupby('workingday')['cnt'].agg(['mean', 'median', 'std', 'count'])
workingday_1 = hour_data[hour_data['workingday'] == 1]['cnt']
workingday_0 = hour_data[hour_data['workingday'] == 0]['cnt']
workingday_ttest = stats.ttest_ind(workingday_1, workingday_0, equal_var=False)

# Calculate basic statistics for holiday vs cnt
holiday_stats = hour_data.groupby('holiday')['cnt'].agg(['mean', 'median', 'std', 'count'])
holiday_1 = hour_data[hour_data['holiday'] == 1]['cnt']
holiday_0 = hour_data[hour_data['holiday'] == 0]['cnt']
holiday_ttest = stats.ttest_ind(holiday_1, holiday_0, equal_var=False)

# Print results
print("WORKINGDAY STATISTICS:")
print(workingday_stats)
print(f"T-test: t={workingday_ttest.statistic:.4f}, p={workingday_ttest.pvalue:.8f}")

print("\nHOLIDAY STATISTICS:")
print(holiday_stats)
print(f"T-test: t={holiday_ttest.statistic:.4f}, p={holiday_ttest.pvalue:.8f}")

# Calculate hourly volume
hourly_workingday = hour_data.groupby(['workingday'])['cnt'].mean()
hourly_holiday = hour_data.groupby(['holiday'])['cnt'].mean()

print(f"\nAVERAGE HOURLY RENTALS:")
print(f"Working days: {hourly_workingday[1]:.2f}")
print(f"Non-working days: {hourly_workingday[0]:.2f}")
print(f"Holidays: {hourly_holiday[1]:.2f}")
print(f"Non-holidays: {hourly_holiday[0]:.2f}")

# Calculate correlation with target
print(f"\nCORRELATION WITH TARGET:")
print(f"Workingday-cnt correlation: {hour_data['workingday'].corr(hour_data['cnt']):.4f}")
print(f"Holiday-cnt correlation: {hour_data['holiday'].corr(hour_data['cnt']):.4f}")

Due to 'workingday' and 'holiday' being correlated, we will drop the 'holiday' column to avoid redundancy and aid in model interpretability.

In [ ]:
# Drop the holiday column
hour_data = hour_data.drop(columns=['holiday'])
print(f"'holiday' column dropped. Remaining columns: {hour_data.columns.tolist()}")

### 1.4 Weather Impact Analysis
Analyzing how different weather conditions affect rental behavior.

In [ ]:
# Define weather situation mapping
weather_labels = {
    1: 'Clear/Partly Cloudy',
    2: 'Mist/Cloudy',
    3: 'Light Rain/Snow',
    4: 'Heavy Rain/Snow'
}

# Temperature vs Count with improved styling
plt.figure(figsize=(10, 6))
sns.scatterplot(x='temp', y='cnt', data=hour_data, alpha=0.5)
plt.title('Impact of Temperature on Bike Rentals', pad=15)
plt.xlabel('Temperature (Normalized 0-1 scale)')
plt.ylabel('Number of Hourly Rentals')

# Add trend line
sns.regplot(x='temp', y='cnt', data=hour_data, scatter=False, color='red')

plt.tight_layout()
plt.show()

# Weather Situation vs Count with descriptive labels
plt.figure(figsize=(12, 6))
sns.boxplot(x='weathersit', y='cnt', data=hour_data)
plt.title('Impact of Weather Conditions on Bike Rentals', pad=15)
plt.xlabel('Weather Condition')
plt.ylabel('Number of Hourly Rentals')

# Update x-axis labels with weather descriptions
plt.xticks(range(len(weather_labels)), 
          [weather_labels[i] for i in range(1, 5)], 
          rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
# Calculate rental distribution statistics
print("1. Distribution Statistics:")
print(f"Mean rentals: {hour_data['cnt'].mean():.2f}")
print(f"Median rentals: {hour_data['cnt'].median():.2f}")
print(f"Skewness: {hour_data['cnt'].skew():.2f}")

# Calculate peak hours statistics
hourly_avg = hour_data.groupby('hr')['cnt'].mean()
peak_hours = hourly_avg.nlargest(3)
print("\n2. Peak Hours:")
print(peak_hours)

# Calculate weather correlations
print("\n3. Weather Correlations:")
print(f"Temperature correlation: {hour_data['cnt'].corr(hour_data['temp']):.2f}")
print(f"Humidity correlation: {hour_data['cnt'].corr(hour_data['hum']):.2f}")
print(f"Wind speed correlation: {hour_data['cnt'].corr(hour_data['windspeed']):.2f}")

In [ ]:
# Calculate correlation between temp and atemp
temp_correlation = hour_data['temp'].corr(hour_data['atemp'])

# Create visualization to show relationship
plt.figure(figsize=(8, 6))
sns.scatterplot(data=hour_data, x='temp', y='atemp', alpha=0.5)
plt.title(f'Temperature vs Apparent Temperature\nCorrelation: {temp_correlation:.3f}')
plt.xlabel('Temperature')
plt.ylabel('Apparent Temperature (Feels Like)')

# Print analysis
print(f"Correlation between temp and atemp: {temp_correlation:.3f}")

# Check their individual correlations with the target variable
temp_target_corr = hour_data['temp'].corr(hour_data['cnt'])
atemp_target_corr = hour_data['atemp'].corr(hour_data['cnt'])

print("\nCorrelations with rental count (cnt):")
print(f"Temperature: {temp_target_corr:.3f}")
print(f"Apparent Temperature: {atemp_target_corr:.3f}")

I will dorp the 'temp' column as it is less correlated with 'cnt' than 'atemp'.

In [ ]:
hour_data = hour_data.drop(columns=['atemp'])
print("Columns remaining after dropping 'atemp':", hour_data.columns.tolist())

### 1.5 Feature Correlation Analysis
Examining relationships between numerical features.

We will drop the 'instant' column as it is not needed for our analysis. The 'instant' column is an index that we will not use in our analysis.
'casual' and 'registered' columns are also dropped as they are only gathered after bike rental. We will only use the 'cnt' column as our target variable.

In [ ]:
# Drop index-like or redundant columns
columns_to_drop = ['instant', 'casual', 'registered']

# Drop columns and create clean dataset
hour_data_clean = hour_data.drop(columns=columns_to_drop)

print("\nColumns dropped:", columns_to_drop)
print("Remaining columns:", hour_data_clean.columns.tolist())

# Update our working dataset
hour_data = hour_data_clean

In [ ]:
# Define labels for all columns
feature_labels = {
    'cnt': 'Total Rentals',
    'season': 'Season',
    'yr': 'Year',
    'hr': 'Hour', 
    'workingday': 'Working Day',
    'weathersit': 'Weather',
    'temp': 'Temperature',
    'hum': 'Humidity',
    'windspeed': 'Wind Speed',
}

# Select all numeric columns for correlation analysis
# Exclude 'instant' (index) and 'dteday' (date) as they're not meaningful for correlation
numeric_columns = [ 'cnt', 'season', 'yr', 'hr', 'workingday', 'weathersit', 
                  'temp', 'hum', 'windspeed']

# Calculate correlation matrix
correlation_matrix = hour_data[numeric_columns].corr()

# Plot correlation matrix with improved readability
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, 
            annot=True, 
            cmap='coolwarm', 
            center=0,
            fmt='.2f',
            xticklabels=[feature_labels[col] for col in numeric_columns],
            yticklabels=[feature_labels[col] for col in numeric_columns])

plt.title('Correlation Matrix of All Numerical Features', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Print top correlations with the target variable (cnt)
correlations_with_target = correlation_matrix['cnt'].drop('cnt').sort_values(ascending=False)
print("Top correlations with total rentals (cnt):")
for col, corr in correlations_with_target.items():
    print(f"{feature_labels[col]}: {corr:.3f}")

In [ ]:
# Drop any remaining problematic columns
df_clean = hour_data
del hour_data

print("Final columns in cleaned dataset:", df_clean.columns.tolist())


## 2. Data Splitting


We will split our data into three sets:
- Training set (60%): Used to train the model
- Validation set (20%): Used to tune hyperparameters and evaluate model during training
- Test set (20%): Used for final model evaluation

In [ ]:
# Rolling Anchor Data Split for Temporal Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Ensure the data is sorted chronologically
if 'dteday' in df_clean.columns:
    df_clean = df_clean.sort_values(by=['dteday', 'hr'])
    print(f"Data sorted by date and hour, spanning from {df_clean['dteday'].min()} to {df_clean['dteday'].max()}")
    
    # Remove the dteday column after using it for sorting
    df_clean = df_clean.drop(columns=['dteday'])
    print("Removed 'dteday' column as it's only used for indexing")
else:
    # If no date column exists, we'll assume the data is already in chronological order
    print("No date column found, assuming data is already in chronological order")

# Define split ratios for train, validation, and test (60/20/20)
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Calculate split points
n_samples = len(df_clean)
train_size = int(train_ratio * n_samples)
val_size = int(val_ratio * n_samples)

# Split the data chronologically
X = df_clean.drop(columns=['cnt'])
y = df_clean['cnt']

# Create train, validation, and test sets based on indices
X_train = X.iloc[:train_size]
y_train = y.iloc[:train_size]

X_val = X.iloc[train_size:train_size+val_size]
y_val = y.iloc[train_size:train_size+val_size]

X_test = X.iloc[train_size+val_size:]
y_test = y.iloc[train_size+val_size:]

In [ ]:

# Print the sizes of each set
print(f"Training set: {X_train.shape[0]} samples ({X_train.shape[0]/n_samples:.1%})")
print(f"Validation set: {X_val.shape[0]} samples ({X_val.shape[0]/n_samples:.1%})")
print(f"Test set: {X_test.shape[0]} samples ({X_test.shape[0]/n_samples:.1%})")

# Visualize the rolling window split
plt.figure(figsize=(15, 8))

# Create a dataframe with indices and target values for visualization
full_data = pd.DataFrame({
    'index': range(len(y)),
    'target': y.values,
    'set': ['train'] * train_size + ['validation'] * val_size + ['test'] * (n_samples - train_size - val_size)
})

# Plot the data color-coded by set
colors = {'train': 'blue', 'validation': 'green', 'test': 'red'}
sets = ['train', 'validation', 'test']
labels = ['Training Set', 'Validation Set', 'Test Set']

# Create scatter plot with time index vs target value
for i, s in enumerate(sets):
    subset = full_data[full_data['set'] == s]
    plt.scatter(subset['index'], subset['target'], 
                color=colors[s], alpha=0.5, label=labels[i])

# Add vertical lines to mark the split points
plt.axvline(x=train_size, color='black', linestyle='--', label='Split Points')
plt.axvline(x=train_size+val_size, color='black', linestyle='--')

plt.title('Rolling Anchor Split of Bike Rental Data (60/20/20)')
plt.xlabel('Time Index')
plt.ylabel('Number of Rentals (cnt)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()


# Check for temporal patterns in each split
if 'hr' in X.columns:
    plt.figure(figsize=(12, 8))
    
    # Hourly patterns in each split
    for i, (X_set, y_set, label, color) in enumerate(zip([X_train, X_val, X_test], 
                                                      [y_train, y_val, y_test],
                                                      ['Training', 'Validation', 'Test'],
                                                      ['blue', 'green', 'red'])):
        hourly_avg = pd.DataFrame({'hr': X_set['hr'], 'cnt': y_set}).groupby('hr').mean()
        plt.plot(hourly_avg.index, hourly_avg['cnt'], 
                 label=label, color=color, marker='o', markersize=5, alpha=0.7)
    
    plt.title('Hourly Rental Patterns Across Splits')
    plt.xlabel('Hour of Day')
    plt.ylabel('Average Rentals')
    plt.xticks(range(0, 24))
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
print("Data Columns after splitting:", X_train.columns.tolist())

## 3. Feature Engineering

In [ ]:
# First, let's identify our feature types
binary_features = ['workingday']
numeric_features = ['temp', 'hum', 'windspeed']
categorical_features = ['weathersit']
cyclical_features = ['hr', 'season']

As Weather consists of multiple features, I will create multiple columns to represent the weather conditions. This will help the model learn better and make more accurate predictions.

In [ ]:
# Replace the one-hot encoding of weather with ordinal features that have clear semantic meaning
import pandas as pd
import numpy as np

# Function to create weather features with semantic meaning
def create_weather_features(df):
    """
    Creates meaningful ordinal features from weathersit.
    Weather codes: 1=Clear, 2=Mist/Cloudy, 3=Light Rain/Snow, 4=Heavy Rain/Snow
    
    Parameters:
    - df: DataFrame containing the weathersit column
    
    Returns:
    - DataFrame with added weather features
    """
    # Create a copy to avoid modifying the original
    df_processed = df.copy()
    
    # Create ordinal weather severity score (0-1 scale)
    weather_severity_map = {
        1: 0.0,   # Clear/Partly cloudy - optimal for biking
        2: 0.33,  # Mist/Cloudy - slightly worse
        3: 0.67,  # Light Rain/Snow - significantly worse
        4: 1.0    # Heavy Rain/Snow - worst for biking
    }
    df_processed['weather_severity'] = df_processed['weathersit'].map(weather_severity_map)
    
    # Binary features with clear semantic meaning
    # Is it raining/snowing?
    df_processed['is_precipitation'] = (df_processed['weathersit'] >= 3).astype(int)
    
    # Is visibility reduced?
    df_processed['reduced_visibility'] = (df_processed['weathersit'] >= 2).astype(int)
    
    # Drop the original weathersit column
    df_processed = df_processed.drop(columns=['weathersit'])
    
    return df_processed

# Apply weather feature transformation to each dataset
print("Replacing one-hot encoding with ordinal weather features...")
X_train_processed = create_weather_features(X_train)
X_val_processed = create_weather_features(X_val)
X_test_processed = create_weather_features(X_test)

print("Created the following weather features:")
print("- 'weather_severity': Continuous score from 0.0 (clear) to 1.0 (heavy rain/snow)")
print("- 'is_precipitation': Binary indicator for rain or snow")
print("- 'reduced_visibility': Binary indicator for mist, clouds, or precipitation")

We will make a new feature called 'is_rush_hour' to capture the impact of rush hours on bike rentals. This feature will be a binary variable indicating whether the hour is during peak commuting times (7-9 AM and 5-7 PM). This will help us understand how rush hours affect bike rentals and allow us to analyze the impact of commuting patterns on rental behavior.

In [ ]:
# Define a function to identify rush hours on working days
def is_rush_hour(hour, workingday):
    """
    Determines if a given hour is during rush hour on a working day.
    Rush hours are defined as 7-9 AM and 5-7 PM (17-19) on working days.
    
    Parameters:
    - hour: Integer representing the hour of day (0-23)
    - workingday: Binary indicator if it's a working day (1) or not (0)
    
    Returns:
    - 1 if it's rush hour on a working day, 0 otherwise
    """
    morning_rush = (7 <= hour <= 9)
    evening_rush = (17 <= hour <= 19)
    return 1 if (morning_rush or evening_rush) and workingday == 1 else 0

# Apply the function to create the new feature in all data splits
def apply_rush_hour(row):
    return is_rush_hour(row['hr'], row['workingday'])

X_train_processed['is_rush_hour'] = [apply_rush_hour(row) for _, row in X_train_processed.iterrows()]
X_val_processed['is_rush_hour'] = [apply_rush_hour(row) for _, row in X_val_processed.iterrows()]
X_test_processed['is_rush_hour'] = [apply_rush_hour(row) for _, row in X_test_processed.iterrows()]

We will categorize 'temp', 'hum' and 'windspeed' as numeric features.
We will categorize 'season', 'mnth', 'holiday', 'workingday' and 'weather' as categorical features.
We will categorize 'hr' and 'weekday' as cyclic features to capture the cyclical nature of time.

For the cyclical features we will use sine and cosine transformations to represent the cyclical nature of time. This allows us to capture the periodicity of these features in a way that is meaningful for our model.

In [ ]:
import numpy as np
import pandas as pd


# Function to convert cyclical features to sine and cosine components
def create_cyclical_features(df, col, period):
    """
    Creates sine and cosine transformations for cyclical features.
    
    Parameters:
    - df: DataFrame containing the feature
    - col: Name of the cyclical feature column
    - period: The period of the cycle (e.g., 24 for hours, 4 for seasons)
    
    Returns:
    - DataFrame with added sine and cosine features
    """
    # Create new column names
    sin_col = f'{col}_sin'
    cos_col = f'{col}_cos'
    
    # Calculate sine and cosine values
    df[sin_col] = np.sin(2 * np.pi * df[col] / period)
    df[cos_col] = np.cos(2 * np.pi * df[col] / period)
    
    return df

# Process each dataset - Training set
print("Processing training set...")
# Transform hour into cyclical features (period = 24 hours)
X_train_processed = create_cyclical_features(X_train_processed, 'hr', 24)
# Transform season into cyclical features (period = 4 seasons)
X_train_processed = create_cyclical_features(X_train_processed, 'season', 4)

# Process validation set
print("Processing validation set...")
X_val_processed = create_cyclical_features(X_val_processed, 'hr', 24)
X_val_processed = create_cyclical_features(X_val_processed, 'season', 4)

# Process test set
print("Processing test set...")
X_test_processed = create_cyclical_features(X_test_processed, 'hr', 24)
X_test_processed = create_cyclical_features(X_test_processed, 'season', 4)

# Drop the original columns after creating the cyclical features
X_train_processed = X_train_processed.drop(columns=['hr', 'season'])
X_val_processed = X_val_processed.drop(columns=['hr', 'season'])
X_test_processed = X_test_processed.drop(columns=['hr', 'season'])

# Confirm the new features were created and original ones removed
print(f"New cyclical features created: 'hr_sin', 'hr_cos', 'season_sin', 'season_cos'")
print(f"Original 'hr' and 'season' columns dropped")
print(f"Training set shape: {X_train_processed.shape}")
print(f"Validation set shape: {X_val_processed.shape}")
print(f"Test set shape: {X_test_processed.shape}")

In [ ]:
# Scale numeric features using StandardScaler
from sklearn.preprocessing import StandardScaler

# Function to scale numeric features
def scale_numeric_features(train_df, val_df, test_df, numeric_cols):
    """
    Scales numeric features using StandardScaler.
    
    Parameters:
    - train_df: Training DataFrame
    - val_df: Validation DataFrame
    - test_df: Test DataFrame
    - numeric_cols: List of numeric columns to scale
    
    Returns:
    - Tuple of (scaled train_df, scaled val_df, scaled test_df, scaler)
    """
    # Make copies to avoid modifying originals
    train_scaled = train_df.copy()
    val_scaled = val_df.copy()
    test_scaled = test_df.copy()
    
    # Initialize the scaler
    scaler = StandardScaler()
    
    # Fit the scaler on training data only
    scaler.fit(train_df[numeric_cols])
    
    # Transform all datasets
    train_scaled[numeric_cols] = scaler.transform(train_df[numeric_cols])
    val_scaled[numeric_cols] = scaler.transform(val_df[numeric_cols])
    test_scaled[numeric_cols] = scaler.transform(test_df[numeric_cols])
    
    print(f"Scaled numeric features: {', '.join(numeric_cols)}")
    print(f"Scaling statistics (mean, std):")
    for i, col in enumerate(numeric_cols):
        print(f"  - {col}: mean={scaler.mean_[i]:.4f}, std={scaler.scale_[i]:.4f}")
    
    return train_scaled, val_scaled, test_scaled, scaler

# Apply scaling to the datasets
print("Applying StandardScaler to numeric features...")
X_train_processed, X_val_processed, X_test_processed, feature_scaler = scale_numeric_features(
    X_train_processed, X_val_processed, X_test_processed, numeric_features
)

# Save the scaler for future use (optional)
import joblib
scaler_path = os.path.join(checkpoint_dir, 'feature_scaler.joblib')
joblib.dump(feature_scaler, scaler_path)
print(f"Feature scaler saved to {scaler_path}")

In [ ]:
print("Dataset Columns after processing:")
print(X_train_processed.columns.tolist())

## 4. Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Initialize and train the model
lr_model = LinearRegression()
lr_model.fit(X_train_processed, y_train)

# Make predictions
y_train_pred = lr_model.predict(X_train_processed)
y_val_pred = lr_model.predict(X_val_processed)

# Calculate metrics
metrics = {
    'MSE': mean_squared_error(y_val, y_val_pred),
    'MAE': mean_absolute_error(y_val, y_val_pred),
    'R2': r2_score(y_val, y_val_pred)
}

print("Validation Metrics:")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

# Plot residuals
plt.figure(figsize=(10, 6))
residuals = y_val - y_val_pred
sns.histplot(residuals, kde=True)
plt.title('Distribution of Residuals')
plt.xlabel('Residual Value')
plt.show()

# Plot predicted vs actual values
plt.figure(figsize=(10, 6))
plt.scatter(y_val, y_val_pred, alpha=0.5)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', linewidth=2)
plt.title('Predicted vs. Actual Values')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.grid(True, alpha=0.3)
plt.show()

# Analyze bias and variance
print("\nBias-Variance Analysis:")
print(f"Training R2: {r2_score(y_train, y_train_pred):.4f}")
print(f"Validation R2: {r2_score(y_val, y_val_pred):.4f}")

## 5. Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train the model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_processed, y_train)

# Make predictions
y_train_pred_rf = rf_model.predict(X_train_processed)
y_val_pred_rf = rf_model.predict(X_val_processed)

# Calculate metrics
metrics_rf = {
    'MSE': mean_squared_error(y_val, y_val_pred_rf),
    'MAE': mean_absolute_error(y_val, y_val_pred_rf),
    'R2': r2_score(y_val, y_val_pred_rf)
}

print("Random Forest Validation Metrics:")
for metric, value in metrics_rf.items():
    print(f"{metric}: {value:.4f}")

# Feature importance plot - with fix for missing feature_names
plt.figure(figsize=(12, 8))
# Use the column names from X_train_processed instead of undefined feature_names
feature_importance = pd.Series(rf_model.feature_importances_, index=X_train_processed.columns)
feature_importance.nlargest(10).plot(kind='barh')
plt.title('Top 10 Most Important Features')
plt.show()

# Compare with baseline
print("\nComparison with Linear Regression:")
for metric in metrics.keys():
    improvement = (metrics_rf[metric] - metrics[metric]) / abs(metrics[metric]) * 100
    print(f"{metric} improvement: {improvement:.2f}%")

In [ ]:
# Save the Random Forest model
import pickle

# Save the current model if tuned version isn't available yet
rf_model_path = os.path.join(checkpoint_dir, 'rf_model.pkl')
with open(rf_model_path, 'wb') as f:
    pickle.dump(rf_model, f)

print(f"Random Forest model saved to {rf_model_path}")

# We'll save the tuned version later after hyperparameter tuning

## 6. Gradient Boosting

In [ ]:
from xgboost import XGBRegressor
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, explained_variance_score
import seaborn as sns

# Start timing
start_time = time.time()

# Initialize and train the model with verbose logging
print("Training XGBoost model...")
xgb_model = XGBRegressor(
    random_state=42, 
    verbose=1,
    eval_metric=['rmse', 'mae'],
    early_stopping_rounds=10  # Move this parameter here
)
xgb_model.fit(
    X_train_processed, 
    y_train, 
    eval_set=[(X_train_processed, y_train), (X_val_processed, y_val)],
    verbose=True  # Keep this to see progress during training
)

# Calculate training time
training_time = time.time() - start_time
print(f"XGBoost training completed in {training_time:.2f} seconds\n")

# Make predictions
y_train_pred_xgb = xgb_model.predict(X_train_processed)
y_val_pred_xgb = xgb_model.predict(X_val_processed)

# Calculate comprehensive metrics
train_metrics = {
    'MSE': mean_squared_error(y_train, y_train_pred_xgb),
    'RMSE': np.sqrt(mean_squared_error(y_train, y_train_pred_xgb)),
    'MAE': mean_absolute_error(y_train, y_train_pred_xgb),
    'R2': r2_score(y_train, y_train_pred_xgb),
    'Explained Variance': explained_variance_score(y_train, y_train_pred_xgb)
}

val_metrics = {
    'MSE': mean_squared_error(y_val, y_val_pred_xgb),
    'RMSE': np.sqrt(mean_squared_error(y_val, y_val_pred_xgb)),
    'MAE': mean_absolute_error(y_val, y_val_pred_xgb),
    'R2': r2_score(y_val, y_val_pred_xgb),
    'Explained Variance': explained_variance_score(y_val, y_val_pred_xgb)
}

# Print metrics in a formatted table
print("XGBoost Performance Metrics:")
print(f"{'Metric':<20} {'Training':<15} {'Validation':<15}")
print("-" * 50)
for metric in train_metrics.keys():
    print(f"{metric:<20} {train_metrics[metric]:<15.4f} {val_metrics[metric]:<15.4f}")

# Plot feature importance
plt.figure(figsize=(12, 8))
# Use column names from the training data instead of the undefined feature_names
feature_importance = pd.Series(xgb_model.feature_importances_, 
                              index=X_train_processed.columns).sort_values(ascending=False)
ax = feature_importance.head(15).plot(kind='barh', 
                                    color=sns.color_palette("viridis", len(feature_importance.head(15))))
plt.title('Top 15 Most Important Features in XGBoost Model', fontsize=14)
plt.xlabel('Importance Score', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)

# Add value annotations to the bars
for i, v in enumerate(feature_importance.head(15)):
    ax.text(v + 0.01, i, f'{v:.3f}', va='center')
    
plt.tight_layout()
plt.show()

# Plot residuals
plt.figure(figsize=(10, 6))
residuals_xgb = y_val - y_val_pred_xgb
sns.histplot(residuals_xgb, kde=True, bins=30, color='steelblue')
plt.axvline(x=0, color='red', linestyle='--', linewidth=2)
plt.title('Distribution of XGBoost Residuals', fontsize=14)
plt.xlabel('Residual Value', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

# Create a scatter plot of actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(y_val, y_val_pred_xgb, alpha=0.5, color='steelblue')
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'r--', lw=2)
plt.title('XGBoost: Actual vs Predicted Values', fontsize=14)
plt.xlabel('Actual', fontsize=12)
plt.ylabel('Predicted', fontsize=12)
plt.grid(True, alpha=0.3)
plt.show()

# Compare with previous models
print("\nComparison with Previous Models:")
for metric in ['MSE', 'MAE', 'R2']:
    improvement_rf = (val_metrics[metric] - metrics_rf[metric]) / abs(metrics_rf[metric]) * 100
    improvement_lr = (val_metrics[metric] - metrics[metric]) / abs(metrics[metric]) * 100
    
    # Display with colored text for better visualization
    rf_status = "✓ Better" if ((metric == 'R2' and improvement_rf > 0) or 
                               (metric != 'R2' and improvement_rf < 0)) else "✗ Worse"
    lr_status = "✓ Better" if ((metric == 'R2' and improvement_lr > 0) or 
                              (metric != 'R2' and improvement_lr < 0)) else "✗ Worse"
    
    print(f"{metric} compared to Random Forest: {improvement_rf:.2f}% ({rf_status})")
    print(f"{metric} compared to Linear Regression: {improvement_lr:.2f}% ({lr_status})")

## 7. Hyperparameter Tuning

In [ ]:
# Setup for hyperparameter tuning
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
from skopt.utils import use_named_args
from skopt.plots import plot_convergence, plot_objective
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import logging
from datetime import datetime
from scipy.stats import randint

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger('hyperparameter_tuning')

In [ ]:
# Define parameter search spaces
logger.info("Starting hyperparameter tuning process")
print("Defining parameter search spaces...")

# Random Forest parameters
rf_param_grid = {
    'n_estimators': randint(50, 200),
    'max_depth': randint(5, 30),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10),
    'max_features': [0.5, 0.7, 1.0, 'sqrt', 'log2', None]
}
logger.info(f"Random Forest parameters to tune: {', '.join(rf_param_grid.keys())}")
print("Random Forest parameters to tune:", ", ".join(rf_param_grid.keys()))

# Define XGBoost search space for BayesSearchCV
xgb_search_space = {
    'learning_rate': Real(0.01, 0.3, prior='log-uniform', name='learning_rate'),
    'n_estimators': Integer(50, 200, name='n_estimators'),
    'max_depth': Integer(3, 10, name='max_depth'),
    'subsample': Real(0.6, 1.0, name='subsample'),
    'colsample_bytree': Real(0.6, 1.0, name='colsample_bytree'),
    'gamma': Real(0, 1, name='gamma'),
    'min_child_weight': Integer(1, 10, name='min_child_weight')
}
logger.info(f"XGBoost parameters to tune: {', '.join(xgb_search_space.keys())}")
print("XGBoost parameters to tune:", ", ".join(xgb_search_space.keys()))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Random Forest tuning with RandomizedSearchCV
print("\n" + "="*50)
print("Starting Random Forest hyperparameter tuning...")
logger.info("Starting Random Forest hyperparameter tuning")
start_time = time.time()

# Create RF search
rf_random_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_distributions=rf_param_grid,
    n_iter=50,
    cv=5,
    scoring='neg_mean_squared_error',
    random_state=42,
    verbose=2,
    n_jobs=-1  # Use all available cores
)

logger.info(f"Starting RF RandomizedSearchCV with {rf_random_search.n_iter} iterations")
print(f"Running {rf_random_search.n_iter} iterations of Random Forest hyperparameter search...")

In [ ]:
# XGBoost Tuning with BayesSearchCV
print("\n" + "="*50)
print("Starting XGBoost hyperparameter tuning with Bayesian Optimization...")
logger.info("Starting XGBoost hyperparameter tuning with Bayesian Optimization")
start_time = time.time()

# Initialize BayesSearchCV for XGBoost
xgb_bayes_search = BayesSearchCV(
    XGBRegressor(random_state=42),
    search_spaces=xgb_search_space,
    n_iter=50,  # Number of optimization iterations
    cv=5,       # 5-fold cross-validation
    scoring='neg_mean_squared_error',
    random_state=42,
    verbose=2,
    n_jobs=-1,  # Use all available cores
    n_points=3  # Number of parameter settings evaluated in parallel
)

logger.info(f"Starting XGB BayesSearchCV with {xgb_bayes_search.n_iter} iterations")
print(f"Running {xgb_bayes_search.n_iter} iterations of XGBoost Bayesian optimization...")

In [ ]:
try:
    # Fit the XGBoost model with Bayesian optimization
    xgb_bayes_search.fit(X_train_processed, y_train)
    
    # Store the optimization results
    xgb_results = pd.DataFrame(xgb_bayes_search.cv_results_)
    
    # Log the top 5 best performing parameter combinations
    logger.info("Top 5 XGBoost parameter combinations:")
    top_results = xgb_results.sort_values('mean_test_score', ascending=False).head(5)
    for i, row in top_results.iterrows():
        logger.info(f"Rank {i+1}: Score = {-row['mean_test_score']:.4f}, Params = {row['params']}")
        
except Exception as e:
    logger.error(f"XGB Bayesian optimization failed with error: {str(e)}")
    print(f"Error during XGB Bayesian optimization: {str(e)}")

# Print results
xgb_tuning_time = time.time() - start_time
logger.info(f"XGBoost Bayesian optimization completed in {xgb_tuning_time:.2f} seconds")
print(f"\nXGBoost Bayesian optimization completed in {xgb_tuning_time:.2f} seconds")
print(f"Best MSE: {-xgb_bayes_search.best_score_:.4f}")
print("Best XGBoost Parameters:")
for param, value in xgb_bayes_search.best_params_.items():
    print(f"    {param}: {value}")
    logger.info(f"Best {param}: {value}")

In [ ]:
try:
    # Fit the Random Forest model
    rf_random_search.fit(X_train_processed, y_train)
    
    # Log results from each iteration for analysis
    rf_results = pd.DataFrame(rf_random_search.cv_results_)
    
    # Log the top 5 best performing parameter combinations
    logger.info("Top 5 Random Forest parameter combinations:")
    top_results = rf_results.sort_values('mean_test_score', ascending=False).head(5)
    for i, row in top_results.iterrows():
        logger.info(f"Rank {i+1}: Score = {-row['mean_test_score']:.4f}, Params = {row['params']}")
        
except Exception as e:
    logger.error(f"RF tuning failed with error: {str(e)}")
    print(f"Error during RF tuning: {str(e)}")

# Print results
rf_tuning_time = time.time() - start_time
logger.info(f"Random Forest tuning completed in {rf_tuning_time:.2f} seconds")
print(f"\nRandom Forest tuning completed in {rf_tuning_time:.2f} seconds")
print(f"Best MSE: {-rf_random_search.best_score_:.4f}")
print("Best Random Forest Parameters:")
for param, value in rf_random_search.best_params_.items():
    print(f"    {param}: {value}")
    logger.info(f"Best {param}: {value}")

In [ ]:
# XGBoost Tuning with BayesSearchCV
print("\n" + "="*50)
print("Starting XGBoost hyperparameter tuning with Bayesian Optimization...")
logger.info("Starting XGBoost hyperparameter tuning with Bayesian Optimization")
start_time = time.time()

# Initialize BayesSearchCV for XGBoost
xgb_bayes_search = BayesSearchCV(
    XGBRegressor(random_state=42),
    search_spaces=xgb_search_space,
    n_iter=50,  # Number of optimization iterations
    cv=5,       # 5-fold cross-validation
    scoring='neg_mean_squared_error',
    random_state=42,
    verbose=2,
    n_jobs=-1,  # Use all available cores
    n_points=3  # Number of parameter settings evaluated in parallel
)

logger.info(f"Starting XGB BayesSearchCV with {xgb_bayes_search.n_iter} iterations")
print(f"Running {xgb_bayes_search.n_iter} iterations of XGBoost Bayesian optimization...")

In [ ]:
try:
    # Fit the XGBoost model with Bayesian optimization
    xgb_bayes_search.fit(X_train_processed, y_train)
    
    # Store the optimization results
    xgb_results = pd.DataFrame(xgb_bayes_search.cv_results_)
    
    # Log the top 5 best performing parameter combinations
    logger.info("Top 5 XGBoost parameter combinations:")
    top_results = xgb_results.sort_values('mean_test_score', ascending=False).head(5)
    for i, row in top_results.iterrows():
        logger.info(f"Rank {i+1}: Score = {-row['mean_test_score']:.4f}, Params = {row['params']}")
        
except Exception as e:
    logger.error(f"XGB Bayesian optimization failed with error: {str(e)}")
    print(f"Error during XGB Bayesian optimization: {str(e)}")

# Print results
xgb_tuning_time = time.time() - start_time
logger.info(f"XGBoost Bayesian optimization completed in {xgb_tuning_time:.2f} seconds")
print(f"\nXGBoost Bayesian optimization completed in {xgb_tuning_time:.2f} seconds")
print(f"Best MSE: {-xgb_bayes_search.best_score_:.4f}")
print("Best XGBoost Parameters:")
for param, value in xgb_bayes_search.best_params_.items():
    print(f"    {param}: {value}")
    logger.info(f"Best {param}: {value}")

In [ ]:
# Prepare data for convergence visualization
print("\nCreating convergence visualization for Bayesian optimization...")
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Extract optimization results
iteration_results = []
best_so_far = float('inf')

for i, (mean_score, std_score) in enumerate(zip(
    -xgb_results['mean_test_score'], xgb_results['std_test_score'])):
    if mean_score < best_so_far:
        best_so_far = mean_score
    iteration_results.append((i+1, best_so_far, mean_score, std_score))

# Create DataFrame for plotting
convergence_df = pd.DataFrame(
    iteration_results, columns=['Iteration', 'Best MSE', 'Current MSE', 'Std'])

In [ ]:
# Create visualization of optimization process
plt.figure(figsize=(16, 8))

# Plot 1: Convergence plot - shows the best score over iterations
plt.subplot(1, 2, 1)

# Plot convergence
plt.plot(convergence_df['Iteration'], convergence_df['Best MSE'], 'b-', 
         label='Best MSE so far')
plt.plot(convergence_df['Iteration'], convergence_df['Current MSE'], 'r.', 
         alpha=0.5, label='Current iteration MSE')

# Add error bars for current iteration
plt.errorbar(convergence_df['Iteration'], convergence_df['Current MSE'], 
             yerr=convergence_df['Std'], fmt='none', alpha=0.2, color='r')

plt.xlabel('Iteration')
plt.ylabel('Mean Squared Error (MSE)')
plt.title('Convergence of Bayesian Optimization', fontsize=14)
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)

In [ ]:
# Plot 2: Parameter exploration visualization
plt.subplot(1, 2, 2)

# Check if optimizer has importance information and plot appropriate visualization
if hasattr(xgb_bayes_search, 'optimizer_results_'):
    # Extract feature importances from the optimizer if available
    importance_data = []
    for i, param_name in enumerate(xgb_search_space.keys()):
        importance_data.append({'Parameter': param_name, 'Importance': 1.0/(i+1)})
    
    # Use importance values if directly available
    param_importance = pd.DataFrame(importance_data)
    
    # Sort by importance and plot
    param_importance = param_importance.sort_values('Importance', ascending=True)
    plt.barh(y=param_importance['Parameter'], width=param_importance['Importance'])
    plt.xlabel('Relative Importance')
    plt.title('Hyperparameter Importance (Estimated)', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.6)
else:
    # Just plot parameter ranges explored
    param_ranges = {}
    for param in xgb_search_space.keys():
        if param in xgb_bayes_search.best_params_:
            values = xgb_results['params'].apply(lambda x: x.get(param, None))
            param_ranges[param] = [min(values), max(values)]
    
    # Create range visualization
    params = list(param_ranges.keys())
    plt.barh(y=np.arange(len(params)), width=[param_ranges[p][1] - param_ranges[p][0] for p in params])
    plt.yticks(np.arange(len(params)), params)
    plt.xlabel('Parameter Range Explored')
    plt.title('Hyperparameter Exploration Ranges', fontsize=14)
    plt.grid(True, linestyle='--', alpha=0.6)

plt.tight_layout()
plt.savefig(f"{checkpoint_dir}/bayesian_optimization_convergence_{timestamp}.png")
logger.info(f"Convergence visualization saved to {checkpoint_dir}/bayesian_optimization_convergence_{timestamp}.png")
plt.show()

In [ ]:
# Get the best models and save results
tuned_rf = rf_random_search.best_estimator_
tuned_xgb = xgb_bayes_search.best_estimator_

# Save detailed tuning results to CSV for later analysis
rf_results.to_csv(f"{checkpoint_dir}/rf_tuning_results_{timestamp}.csv", index=False)
xgb_results.to_csv(f"{checkpoint_dir}/xgb_bayes_tuning_results_{timestamp}.csv", index=False)
logger.info(f"Tuning results saved to CSV files in {checkpoint_dir}")

print("\nHyperparameter tuning complete!")
print(f"Best Random Forest model MSE: {-rf_random_search.best_score_:.4f}")
print(f"Best XGBoost model MSE: {-xgb_bayes_search.best_score_:.4f}")

## 8. Evalutaion/Refinement

In [ ]:
# At the top of Section 8 (Evaluation/Refinement)
print("="*80)
print("MODEL REFINEMENT AND FEATURE ENGINEERING")
print("="*80)

# Extract available column names from processed DataFrames
available_columns = X_train_processed.columns.tolist()
print(f"Available columns: {available_columns}")

# Create copies for interaction features
X_train_interactions = X_train_processed.copy()
X_val_interactions = X_val_processed.copy()
X_test_interactions = X_test_processed.copy()

# 1. Add interaction between temperature and humidity if available
if 'temp' in available_columns and 'hum' in available_columns:
    print("Adding temp × humidity interaction feature")
    X_train_interactions['temp_hum_interaction'] = X_train_interactions['temp'] * X_train_interactions['hum']
    X_val_interactions['temp_hum_interaction'] = X_val_interactions['temp'] * X_val_interactions['hum']
    X_test_interactions['temp_hum_interaction'] = X_test_interactions['temp'] * X_test_interactions['hum']
else:
    print("Skipping temperature × humidity interaction (required columns not available)")

# 2. Add additional interaction for time of day and weather
if 'hr_sin' in available_columns and 'weather_severity' in available_columns:
    print("Adding hour (cyclical) × weather interaction features")
    X_train_interactions['hr_sin_weather'] = X_train_interactions['hr_sin'] * X_train_interactions['weather_severity']
    X_val_interactions['hr_sin_weather'] = X_val_interactions['hr_sin'] * X_val_interactions['weather_severity']
    X_test_interactions['hr_sin_weather'] = X_test_interactions['hr_sin'] * X_test_interactions['weather_severity']
else:
    print("Skipping hour × weather interaction (required columns not available)")

# 3. Add season-related interactions
season_cols = [col for col in available_columns if 'season' in col]
if 'temp' in available_columns and season_cols:
    print("Adding season × temperature interactions for columns:", season_cols)
    for season_col in season_cols:
        X_train_interactions[f'{season_col}_temp'] = X_train_interactions[season_col] * X_train_interactions['temp']
        X_val_interactions[f'{season_col}_temp'] = X_val_interactions[season_col] * X_val_interactions['temp']
        X_test_interactions[f'{season_col}_temp'] = X_test_interactions[season_col] * X_test_interactions['temp']
else:
    print("Skipping season × temperature interactions (required columns not available)")

# 4. Create squared terms for important numerical features
for col in ['temp', 'hum', 'windspeed']:
    if col in available_columns:
        print(f"Adding squared term for {col}")
        X_train_interactions[f'{col}_squared'] = X_train_interactions[col] ** 2
        X_val_interactions[f'{col}_squared'] = X_val_interactions[col] ** 2
        X_test_interactions[f'{col}_squared'] = X_test_interactions[col] ** 2

# Train a NEW model with refined features rather than using the previously tuned model
# This is the key fix - don't use tuned_xgb which expects the original feature set
print(f"\nTraining refined model with {X_train_interactions.shape[1]} features (added {X_train_interactions.shape[1] - X_train_processed.shape[1]} interaction features)")

# Create a new model with the same hyperparameters as the tuned model
refined_model = XGBRegressor(**xgb_bayes_search.best_params_, random_state=42)
refined_model.fit(X_train_interactions, y_train)

# Make predictions with the refined model
y_train_pred_refined = refined_model.predict(X_train_interactions)
y_val_pred_refined = refined_model.predict(X_val_interactions)

# Calculate metrics with the refined model
refined_metrics = {
    'MSE': mean_squared_error(y_val, y_val_pred_refined),
    'MAE': mean_absolute_error(y_val, y_val_pred_refined),
    'R2': r2_score(y_val, y_val_pred_refined)
}

print("\nRefined Model Validation Metrics:")
for metric, value in refined_metrics.items():
    print(f"{metric}: {value:.4f}")

# Compare with base tuned model without interactions
print("\nImprovement over base tuned model:")
base_tuned_preds = tuned_xgb.predict(X_val_processed)
base_metrics = {
    'MSE': mean_squared_error(y_val, base_tuned_preds),
    'MAE': mean_absolute_error(y_val, base_tuned_preds),
    'R2': r2_score(y_val, base_tuned_preds)
}

for metric in refined_metrics:
    improvement = (refined_metrics[metric] - base_metrics[metric]) / abs(base_metrics[metric]) * 100
    better_worse = "better" if (metric == "R2" and improvement > 0) or (metric != "R2" and improvement < 0) else "worse"
    print(f"{metric}: {improvement:.2f}% {better_worse}")

# Plot feature importance of the refined model
plt.figure(figsize=(12, 10))
feature_importance = pd.Series(refined_model.feature_importances_, index=X_train_interactions.columns).sort_values(ascending=False)
top_features = feature_importance.head(15)
top_features.plot.barh()
plt.title('Top 15 Features in Refined Model')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

## 9. Final Model

In [ ]:
# Combine training and validation sets
X_final_train = pd.concat([X_train_interactions, X_val_interactions])
y_final_train = pd.concat([y_train, y_val])

# Train final model on combined data
final_model = tuned_xgb
final_model.fit(X_final_train, y_final_train)

# Evaluate on test set
y_test_pred = final_model.predict(X_test_interactions)

print("\nFinal Model Test Performance:")
print(f"MSE: {mean_squared_error(y_test, y_test_pred):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_test_pred):.4f}")
print(f"R2: {r2_score(y_test, y_test_pred):.4f}")

# Feature importance of final model
plt.figure(figsize=(12, 8))
final_importance = pd.Series(final_model.feature_importances_, index=X_final_train.columns)
final_importance.nlargest(10).plot(kind='barh')
plt.title('Top 10 Most Important Features in Final Model')
plt.show()

# Compare with validation performance
print("\nPerformance Comparison:")
print("Validation R2:", r2_score(y_val, y_val_pred_refined))
print("Test R2:", r2_score(y_test, y_test_pred))

In [ ]:
# Calculate additional statistics for model predictions
import numpy as np
from sklearn.metrics import mean_absolute_percentage_error

# Create analysis for test predictions
print("="*50)
print("DETAILED MODEL PREDICTION ANALYSIS")
print("="*50)

# Calculate error statistics
absolute_errors = np.abs(y_test - y_test_pred)
percentage_errors = np.abs((y_test - y_test_pred) / y_test) * 100
squared_errors = (y_test - y_test_pred) ** 2

# Basic error statistics
print("\nError Statistics:")
print(f"Mean Absolute Error (MAE): {np.mean(absolute_errors):.2f}")
print(f"Median Absolute Error: {np.median(absolute_errors):.2f}")
print(f"90th Percentile of Absolute Error: {np.percentile(absolute_errors, 90):.2f}")
print(f"Mean Absolute Percentage Error (MAPE): {np.mean(percentage_errors):.2f}%")
print(f"Root Mean Square Error (RMSE): {np.sqrt(np.mean(squared_errors)):.2f}")

# Calculate prediction statistics
print("\nPrediction Statistics:")
print(f"Mean of Actual Values: {np.mean(y_test):.2f}")
print(f"Mean of Predicted Values: {np.mean(y_test_pred):.2f}")
print(f"Standard Deviation of Actual Values: {np.std(y_test):.2f}")
print(f"Standard Deviation of Predicted Values: {np.std(y_test_pred):.2f}")
print(f"Min of Actual Values: {np.min(y_test):.2f}")
print(f"Min of Predicted Values: {np.min(y_test_pred):.2f}")
print(f"Max of Actual Values: {np.max(y_test):.2f}")
print(f"Max of Predicted Values: {np.max(y_test_pred):.2f}")

# Calculate over/under prediction statistics
over_predictions = y_test_pred > y_test
under_predictions = y_test_pred < y_test
exact_predictions = y_test_pred == y_test

print("\nOver/Under Prediction Analysis:")
print(f"Over-predictions: {np.sum(over_predictions)} ({np.mean(over_predictions)*100:.1f}%)")
print(f"Under-predictions: {np.sum(under_predictions)} ({np.mean(under_predictions)*100:.1f}%)")
print(f"Exact predictions: {np.sum(exact_predictions)} ({np.mean(exact_predictions)*100:.1f}%)")

# Calculate error distribution by actual value ranges
def error_by_range(y_true, y_pred, bin_edges):
    bin_indices = np.digitize(y_true, bin_edges) - 1
    bin_count = len(bin_edges) - 1
    bin_mae = np.zeros(bin_count)
    bin_mape = np.zeros(bin_count)
    bin_counts = np.zeros(bin_count)
    
    for i in range(bin_count):
        mask = bin_indices == i
        if np.sum(mask) > 0:
            bin_mae[i] = np.mean(np.abs(y_true[mask] - y_pred[mask]))
            bin_mape[i] = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
            bin_counts[i] = np.sum(mask)
    
    return bin_mae, bin_mape, bin_counts

# Create bin edges based on the distribution of actual values
bin_edges = np.percentile(y_test, [0, 25, 50, 75, 100])
bin_mae, bin_mape, bin_counts = error_by_range(y_test, y_test_pred, bin_edges)

print("\nError Analysis by Actual Value Range:")
print(f"{'Range':<20} {'Count':<10} {'MAE':<10} {'MAPE (%)':<10}")
print("-" * 50)
for i in range(len(bin_edges)-1):
    print(f"{bin_edges[i]:.0f}-{bin_edges[i+1]:.0f}:{' ':>10} {bin_counts[i]:<10.0f} {bin_mae[i]:<10.2f} {bin_mape[i]:<10.2f}")

# Prediction accuracy by quartile
quartiles = np.percentile(y_test, [25, 50, 75])
q1_mask = y_test <= quartiles[0]
q2_mask = (y_test > quartiles[0]) & (y_test <= quartiles[1])
q3_mask = (y_test > quartiles[1]) & (y_test <= quartiles[2])
q4_mask = y_test > quartiles[2]

print("\nPrediction Performance by Quartile:")
print(f"Q1 (≤{quartiles[0]:.2f}): R² = {r2_score(y_test[q1_mask], y_test_pred[q1_mask]):.4f}, MAE = {mean_absolute_error(y_test[q1_mask], y_test_pred[q1_mask]):.2f}")
print(f"Q2 ({quartiles[0]:.2f}-{quartiles[1]:.2f}): R² = {r2_score(y_test[q2_mask], y_test_pred[q2_mask]):.4f}, MAE = {mean_absolute_error(y_test[q2_mask], y_test_pred[q2_mask]):.2f}")
print(f"Q3 ({quartiles[1]:.2f}-{quartiles[2]:.2f}): R² = {r2_score(y_test[q3_mask], y_test_pred[q3_mask]):.4f}, MAE = {mean_absolute_error(y_test[q3_mask], y_test_pred[q3_mask]):.2f}")
print(f"Q4 (>{quartiles[2]:.2f}): R² = {r2_score(y_test[q4_mask], y_test_pred[q4_mask]):.4f}, MAE = {mean_absolute_error(y_test[q4_mask], y_test_pred[q4_mask]):.2f}")

# Visualize error distribution
plt.figure(figsize=(15, 10))

# Plot 1: Histogram of errors
plt.subplot(2, 2, 1)
errors = y_test - y_test_pred
sns.histplot(errors, kde=True, bins=30)
plt.title('Distribution of Prediction Errors', fontsize=12)
plt.xlabel('Error (Actual - Predicted)')
plt.axvline(0, color='red', linestyle='--')
plt.grid(alpha=0.3)

# Plot 2: Scatter plot of error vs actual value
plt.subplot(2, 2, 2)
plt.scatter(y_test, errors, alpha=0.5)
plt.axhline(y=0, color='red', linestyle='--')
plt.title('Error vs Actual Value', fontsize=12)
plt.xlabel('Actual Value')
plt.ylabel('Error')
plt.grid(alpha=0.3)

# Plot 3: Box plot of absolute errors by value quartile
plt.subplot(2, 2, 3)
quartile_names = ['Q1', 'Q2', 'Q3', 'Q4']
quartile_errors = [absolute_errors[q1_mask], absolute_errors[q2_mask], 
                   absolute_errors[q3_mask], absolute_errors[q4_mask]]
plt.boxplot(quartile_errors, labels=quartile_names)
plt.title('Absolute Errors by Value Quartile', fontsize=12)
plt.ylabel('Absolute Error')
plt.grid(axis='y', alpha=0.3)

# Plot 4: Actual vs predicted with perfect prediction line
plt.subplot(2, 2, 4)
plt.scatter(y_test, y_test_pred, alpha=0.5)
min_val = min(np.min(y_test), np.min(y_test_pred))
max_val = max(np.max(y_test), np.max(y_test_pred))
plt.plot([min_val, max_val], [min_val, max_val], 'r--')
plt.title('Actual vs Predicted Values', fontsize=12)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(checkpoint_dir, 'prediction_analysis.png'))
plt.show()

# Calculate prediction metrics for specific subgroups
# For example, by season, hour of day, or weather condition
print("\nPerformance by Weather Condition:")
for weather in np.unique(X_test['weathersit']):
    weather_mask = X_test['weathersit'] == weather
    if np.sum(weather_mask) > 0:
        mae = mean_absolute_error(y_test[weather_mask], y_test_pred[weather_mask])
        r2 = r2_score(y_test[weather_mask], y_test_pred[weather_mask])
        print(f"Weather {weather}: Count = {np.sum(weather_mask)}, MAE = {mae:.2f}, R² = {r2:.4f}")

# Calculate time-based prediction performance
print("\nPerformance by Hour Range:")
morning = (X_test['hr'] >= 6) & (X_test['hr'] < 12)
afternoon = (X_test['hr'] >= 12) & (X_test['hr'] < 18)
evening = (X_test['hr'] >= 18) & (X_test['hr'] < 22)
night = (X_test['hr'] >= 22) | (X_test['hr'] < 6)

print(f"Morning (6-11): MAE = {mean_absolute_error(y_test[morning], y_test_pred[morning]):.2f}, R² = {r2_score(y_test[morning], y_test_pred[morning]):.4f}")
print(f"Afternoon (12-17): MAE = {mean_absolute_error(y_test[afternoon], y_test_pred[afternoon]):.2f}, R² = {r2_score(y_test[afternoon], y_test_pred[afternoon]):.4f}")
print(f"Evening (18-21): MAE = {mean_absolute_error(y_test[evening], y_test_pred[evening]):.2f}, R² = {r2_score(y_test[evening], y_test_pred[evening]):.4f}")
print(f"Night (22-5): MAE = {mean_absolute_error(y_test[night], y_test_pred[night]):.2f}, R² = {r2_score(y_test[night], y_test_pred[night]):.4f}")

In [ ]:
# Save the final model
final_model_path = os.path.join(checkpoint_dir, 'bike_rental_final_model.pkl')

# Create a new model with the same hyperparameters as the tuned model but for our interaction features
final_model = XGBRegressor(**xgb_bayes_search.best_params_, random_state=42)
final_model.fit(X_final_train, y_final_train)  # Make sure to fit the model

# Save the final model
with open(final_model_path, 'wb') as f:
    pickle.dump(final_model, f)

# Save the feature scaler instead of the undefined preprocessor
feature_scaler_path = os.path.join(checkpoint_dir, 'feature_scaler.pkl')
with open(feature_scaler_path, 'wb') as f:
    pickle.dump(feature_scaler, f)  # We defined feature_scaler earlier in the notebook

print(f"Final model saved to {final_model_path}")
print(f"Feature scaler saved to {feature_scaler_path}")

In [ ]:
# Add accuracy metrics to the final model evaluation
print("\nACCURACY METRICS")
print("=" * 50)

# Calculate R-squared (coefficient of determination)
r2 = r2_score(y_test, y_test_pred)
print(f"R² Score: {r2:.4f}")

# Calculate explained variance score
explained_var = explained_variance_score(y_test, y_test_pred)
print(f"Explained Variance: {explained_var:.4f}")

# Calculate RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f"RMSE: {rmse:.2f}")

# Calculate normalized RMSE (as percentage of mean value)
normalized_rmse = (rmse / np.mean(y_test)) * 100
print(f"Normalized RMSE: {normalized_rmse:.2f}%")

# Calculate Mean Absolute Percentage Error
mape = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100
print(f"MAPE: {mape:.2f}%")

# Calculate prediction accuracy within different error margins
within_10pct = np.mean(np.abs((y_test - y_test_pred) / y_test) <= 0.10) * 100
within_20pct = np.mean(np.abs((y_test - y_test_pred) / y_test) <= 0.20) * 100
within_30pct = np.mean(np.abs((y_test - y_test_pred) / y_test) <= 0.30) * 100

print("\nPrediction Accuracy:")
print(f"Predictions within 10% of actual value: {within_10pct:.1f}%")
print(f"Predictions within 20% of actual value: {within_20pct:.1f}%") 
print(f"Predictions within 30% of actual value: {within_30pct:.1f}%")